In [1]:
import boto3
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, accuracy_score

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
## Defining the input and target variables
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Scaling inputs to 0-1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [4]:
## Adaboost
md1 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

## Predicting on the test data set 
pred1 = md1.predict_proba(X_test)[:, 1]
pred1_label = np.where(pred1 < 0.15, 0, 1)

## Computing the accuracy and recall
print('The accuracy is', accuracy_score(Y_test, pred1_label))
print('The recall is', recall_score(Y_test, pred1_label))

The accuracy is 0.17349726775956284
The recall is 0.9910714285714286


In [5]:
## Gradient Boosting 
md2 = GradientBoostingClassifier(n_estimators = 500, max_depth = 3, learning_rate = 0.01).fit(X_train, Y_train)

## Predicting on the test data set 
pred2 = md2.predict_proba(X_test)[:, 1]
pred2_label = np.where(pred2 < 0.15, 0, 1)

## Computing the accuracy and recall
print('The accuracy is', accuracy_score(Y_test, pred2_label))
print('The recall is', recall_score(Y_test, pred2_label))

The accuracy is 0.587431693989071
The recall is 0.8035714285714286


In [6]:
## Support vector machine
md3 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

## Predicting on the test data set 
pred3 = md3.predict_proba(X_test)[:, 1]
pred3_label = np.where(pred3 < 0.15, 0, 1)

## Computing the accuracy and recall
print('The accuracy is', accuracy_score(Y_test, pred3_label))
print('The recall is', recall_score(Y_test, pred3_label))

The accuracy is 0.605191256830601
The recall is 0.5446428571428571


In [16]:
## Ensemble of likelihood
X_rf = pd.concat([pd.DataFrame(pred1), pd.DataFrame(pred2), pd.DataFrame(pred3), Y_test.reset_index(drop = True)], axis = 1)

## Defining the input and target
X_rf_1 = pd.concat([pd.DataFrame(pred1), pd.DataFrame(pred2), pd.DataFrame(pred3)], axis = 1)

## Building the random forest model 
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_rf_1, Y_test)

## Extracting the ensemble likelihood
RF_preds = RF_md.predict_proba(X_rf_1)[:, 1]

## Final results 
final_results = pd.concat([pd.DataFrame(pred1), pd.DataFrame(pred2), pd.DataFrame(pred3), pd.DataFrame(RF_preds), Y_test.reset_index(drop = True)], axis = 1)
final_results.columns = ['AdaBoost_Pred', 'GradienBoost_Pred', 'SVM_Pred', 'RF_ensemble', 'TenYearCHD']
final_results.head(20)

,AdaBoost_Pred,GradienBoost_Pred,SVM_Pred,RF_ensemble,TenYearCHD
0,0.661112,0.559590,0.238198,0.268982,1
1,0.161345,0.083048,0.141316,0.058454,0
2,0.297268,0.067659,0.184040,0.084072,0
3,0.280541,0.067997,0.147728,0.042626,0
4,0.464420,0.606214,0.186765,0.268758,0
5,0.428345,0.182458,0.144356,0.139022,0
6,0.369625,0.079127,0.155035,0.067360,0
7,0.395114,0.095696,0.147687,0.081617,0
8,0.450873,0.325377,0.154476,0.318953,0
9,0.403986,0.124460,0.156591,0.107548,0
